# Random Forest

Random forest is a supervised learning algorithm. The "forest" it builds, is an ensemble of decision trees, usually trained with the “bagging” method. The general idea of the bagging method is that a combination of many "dumb" learning models can create one large "smart" model.

In [1]:
import numpy as np
import math

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston

from scipy.optimize import minimize

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
class TreeEnsemble():
    '''
    Tree ensemble is a class that holds many Decision Trees and uses their combined decision/vote to return a prediction.
    X is a matrix of data values (rows are samples, columns are attributes)
    y is a vector of corresponding target values
    n_trees is the number of trees to create
    sample_sz is the size of the sample set to use of each of the trees in the forest (chose the samples randomly, with or without repetition)
    min_leaf is the minimal number of samples in each leaf node of each tree in the forest
    '''
    def __init__(self, n_trees, sample_sz, min_leaf):
        self.trees = []
        self.n_trees = n_trees
        self.sample_sz = sample_sz
        self.min_leaf = min_leaf
        
    def fit(self, X, y):
        for tree_i in range(self.n_trees):
            tree = DecisionTree(self.sample_sz, self.min_leaf)
            self.trees.append(tree.fit(X, y))
  
    def predict(self, X):
        pred = []
        for tree in self.trees:
            pred.append(tree.predict(X))
        return np.asarray(pred).mean(axis=0)

    def oob_mse(self):
        '''
        compute the mean squared error over all out of bag (oob) samples. That is, for each sample calculate the squared error using  predictions from 
        the trees that do not contain x in their respective bootstrap sample, then average this score for all samples.
        '''
        errors = []
        for tree in self.trees:
            errors.append(tree.oob_mse())
        return np.asarray(errors).mean()

In [4]:
class DecisionTree():
    '''
    A decision tree is a flowchart-like structure in which each internal node represents a question on an attribute (Taller than 1.5 meters?, Black Hair?),
    each branch represents the outcome of the question on that datapoint, and each leaf node represents a class label 
    X is a matrix of data values (rows are samples, columns are attributes)
    y is a vector of corresponding target values
    sample_sz is the size of the sample set to use of each of the trees in the forest (chose the samples randomly, with or without repetition)
    min_leaf is the minimal number of samples in each leaf node of each tree in the forest
    '''
    def __init__(self, sample_sz, min_leaf):
        self.min_leaf = min_leaf
        self.sample_size =  sample_sz

    def fit(self, X, y):
        # sample from X
        num_samples = X.shape[0]
        sample = np.random.randint(0, num_samples, self.sample_size)
        self.X = X[sample]
        self.y = y[sample]
        not_sampled = [i for i in np.arange(num_samples) if i not in sample]
        self.oob_X = X[not_sampled]
        self.oob_y = y[not_sampled]
        # call recursive builder
        self.top_node = Node()
        self.recursive_tree_builder(self.X, self.y, self.top_node)
        
        return self

    def predict(self, X):
        # run on all trees and get proba per classes (make into vector and then avergae columns)
        return np.apply_along_axis(self.predict_single, arr=X, axis=1)
    
    def predict_single(self, x):
        node = self.top_node
        val = node.value
        feat_idx = node.feature
        while(True):
            if node.value is None or node.feature is None:
                return node.mean
            if x[feat_idx] > val:
                node = node.bigger
                val = node.value
                feat_idx = node.feature
            else:
                node = node.smaller
                val = node.value
                feat_idx = node.feature
                
    def oob_mse(self):
         return mean_squared_error(self.predict(self.oob_X), self.oob_y)
    
    def recursive_tree_builder(self, X, y, curr_node):
        #if we have less than min leaf we return
        if X.shape[0] <= 2 * self.min_leaf:
            # update as end node with proba
            curr_node.mean = np.mean(y)
            return
        else:
            # find best feature to split by
            curr_node.feature, curr_node.value = self.best_split(X, y)
            if curr_node.value is None:
                curr_node.mean = np.mean(y)
                return
            bigger = X[:,curr_node.feature] > curr_node.value
            smaller = X[:,curr_node.feature] <= curr_node.value
            self.recursive_tree_builder(X[bigger,:], y[bigger], curr_node.set_bigger())
            self.recursive_tree_builder(X[smaller,:], y[smaller], curr_node.set_smaller())
            return
                
    def best_split(self, X, y):
        # for each feature we check 'all' points and take point with lowest
        kwargs = {'y': y, 'min_leaf':self.min_leaf}
        min_split_per_feature, error = np.apply_along_axis(self.get_min_split, arr=X, axis=0, **kwargs)
        feature = np.argmin(error)
        split_val = min_split_per_feature[feature]
        return feature, split_val
                                   
    def get_min_split(self, feat, y, min_leaf):
        idxs = np.argsort(feat)
        feat = np.sort(feat)
        y = y[idxs]
        bounds = feat[min_leaf: -(min_leaf + 1)]

        min_error = math.inf
        split_val = None
        for trial in bounds:
            if self.bad_trial(trial, feat):
                pass
            error = self.get_var_error(trial, feat, y)
            if error < min_error:
                min_error = error
                split_val = trial
        return (split_val, min_error)

    def bad_trial(self, split, feat):
        bigger = feat > split
        smaller = feat <= split
        return (feat[bigger].shape[0] <= self.min_leaf) or (feat[smaller].shape[0] <= self.min_leaf)
                                
    def get_var_error(self, split, feat, y):
        bigger = feat > split
        smaller = feat <= split

        var_bigger = np.square(np.var(feat[bigger]))
        var_smaller = np.square(np.var(feat[smaller]))
        
        bigger_size = feat[bigger].shape[0]
        smaller_size = feat[smaller].shape[0]
        n = feat.shape[0]

        return (bigger_size/n)*var_bigger + (smaller_size/n)*var_smaller

In [5]:
class Node:
    '''
    This class represents a single node from a DecisionTree.
    '''
    def __init__(self):
        self.feature = None
        self.value = None
        self.smaller = None
        self.bigger = None
        self.mean = None

    def set_bigger(self):
        """
        Creates child, adds to child list and returns child
        """
        self.bigger = Node()
        return self.bigger
                                   
    def set_smaller(self):
        """
        Creates child, adds to child list and returns child
        """
        self.smaller = Node()
        return self.smaller

    def is_leaf(self):
        return self.smaller is None and self.bigger is None
    
    def print_tree(self, depth):
        if self.is_leaf():
             print(f'Probas: {self.mean}')
        else:
            print(f'Node level {depth}, Feature: {self.feature}, Split val: {self.value}')
            self.smaller.print_tree(depth+1)
            self.bigger.print_tree(depth+1)

## Predict Boston Housing data

In [6]:
X, y = load_boston(return_X_y=True)

In [7]:
te = TreeEnsemble(n_trees=2, sample_sz=100, min_leaf=10)

In [8]:
te.fit(X, y)

In [9]:
te.oob_mse()

67.3533754364991

In [10]:
regressor = TreeEnsemble(n_trees=10, sample_sz=100, min_leaf=5)

In [11]:
regressor.fit(X, y)

In [12]:
regressor.oob_mse()

65.81583186544108

# Manual Grid Search

In [ ]:
for n in [1,5,10,20,50,100]:
    for sz in [50,100,300,500]:
        for min_leaf in [1,5, 10]:
            forest = TreeEnsemble(n, sz, min_leaf)
            forest.fit(X, y)
            mse = forest.oob_mse()
            print(f"n_trees:{n}, sz:{sz}, min_leaf:{min_leaf} --- oob mse: {mse}")
            
        print()

n_trees:1, sz:50, min_leaf:1 --- oob mse: 97.9634649122807
n_trees:1, sz:50, min_leaf:5 --- oob mse: 71.90387728472858
n_trees:1, sz:50, min_leaf:10 --- oob mse: 77.28887397660819

n_trees:1, sz:100, min_leaf:1 --- oob mse: 65.8636701556629
n_trees:1, sz:100, min_leaf:5 --- oob mse: 66.93963757416662
n_trees:1, sz:100, min_leaf:10 --- oob mse: 68.47989952046046

n_trees:1, sz:300, min_leaf:1 --- oob mse: 36.197046945469445
n_trees:1, sz:300, min_leaf:5 --- oob mse: 56.80348075025134
n_trees:1, sz:300, min_leaf:10 --- oob mse: 48.988806273653715

n_trees:1, sz:500, min_leaf:1 --- oob mse: 45.48401552287581
n_trees:1, sz:500, min_leaf:5 --- oob mse: 46.47288138847099
n_trees:1, sz:500, min_leaf:10 --- oob mse: 44.26135525815943

n_trees:5, sz:50, min_leaf:1 --- oob mse: 78.31511313582509
n_trees:5, sz:50, min_leaf:5 --- oob mse: 71.12077151475054
n_trees:5, sz:50, min_leaf:10 --- oob mse: 75.63303477311787

n_trees:5, sz:100, min_leaf:1 --- oob mse: 71.62720517247911
n_trees:5, sz:100, m